In [1]:
#configurando o ambiente
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyspark -q

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [4]:
spark = SparkSession.builder.master("local[*]").appName("CVM_Gold").getOrCreate()

In [5]:
path_silver = '/content/drive/MyDrive/pipeline_bigdata_pyspark/silver/fundos_cvm'
df_silver = spark.read.parquet(path_silver)

##📊 Criando Métricas de Analytics

1. Captação Líquida e Performance

Vamos calcular quanto de dinheiro realmente entrou ou saiu do fundo (`CAPTC_LIQUIDA`) e como o patrimônio se comportou.

In [6]:
# Criando a coluna de Captação Líquida
df_gold = df_silver.withColumn("CAPTC_LIQUIDA", F.col("CAPTC_DIA") - F.col("RESG_DIA"))

# Agregando por Fundo para ver os maiores do período
ranking_patrimonio = df_gold.groupBy("CNPJ_FUNDO_CLASSE") \
    .agg(
        F.max("VL_PATRIM_LIQ").alias("PATRIMONIO_MAX"),
        F.sum("CAPTC_LIQUIDA").alias("TOTAL_CAPTC_LIQUIDA"),
        F.avg("NR_COTST").alias("MEDIA_COTISTAS")
    ) \
    .orderBy(F.desc("PATRIMONIO_MAX"))

ranking_patrimonio.show(5)

+------------------+------------------+--------------------+-----------------+
| CNPJ_FUNDO_CLASSE|    PATRIMONIO_MAX| TOTAL_CAPTC_LIQUIDA|   MEDIA_COTISTAS|
+------------------+------------------+--------------------+-----------------+
|07.593.972/0001-86|1.6139562805236E11| 3.709273933859998E9|265.8636363636364|
|42.592.315/0001-15| 1.480798032203E11|   1.457819324062E10|23937.68181818182|
|46.133.770/0001-03|1.4807554972698E11|1.436679997988000...|              1.0|
|00.822.055/0001-87|1.0926699401723E11|          -9.90959E8|              1.0|
|01.597.187/0001-15| 9.301896282606E10|1.4018843994300003E9|             43.0|
+------------------+------------------+--------------------+-----------------+
only showing top 5 rows


2. Evolução Mensal da Indústria (Sazonalidade)

Como o mercado se comporta mês a mês? Quais momentos de crise e euforia?

In [7]:
# Criando colunas de Ano e Mês para agrupamento
df_mensal = df_silver.withColumn("ANO_MES", F.date_format("DT_COMPTC", "yyyy-MM"))

gold_evolucao_mensal = df_mensal.groupBy("ANO_MES") \
    .agg(
        F.sum("VL_PATRIM_LIQ").alias("PATRIMONIO_TOTAL"),
        F.sum("CAPTC_DIA").alias("TOTAL_CAPTACAO"),
        F.sum("RESG_DIA").alias("TOTAL_RESGATE"),
        F.avg("NR_COTST").alias("MEDIA_COTISTAS")
    ).orderBy("ANO_MES")

In [8]:
# Convertendo para Pandas para análise visual rápida
analise_result = gold_evolucao_mensal.toPandas()

# Calculando o Saldo Líquido (O termômetro do mercado)
analise_result['SALDO_LIQUIDO'] = analise_result['TOTAL_CAPTACAO'] - analise_result['TOTAL_RESGATE']

display(analise_result)

,ANO_MES,PATRIMONIO_TOTAL,TOTAL_CAPTACAO,TOTAL_RESGATE,MEDIA_COTISTAS,SALDO_LIQUIDO
0,2024-01,2.410547e+14,1.375382e+12,1.265636e+12,984.225976,1.097461e+11


1. Momento de Euforia (Bull Market):

* SALDO_LIQUIDO fortemente positivo (muito mais captação que resgate).

* O PATRIMONIO_TOTAL sobe de forma consistente mês a mês.

* O número de cotistas (MEDIA_COTISTAS) tende a crescer, indicando novos investidores entrando na festa.

2. Momento de Crise (Bear Market / Panic):

* O SALDO_LIQUIDO fica negativo (os resgates superam as entradas).

* O PATRIMONIO_TOTAL despenca.

Isso acontece por dois motivos: as pessoas estão tirando dinheiro e os ativos que o fundo possui estão desvalorizando.

--> O sinal de alerta: Se o patrimônio cai muito mais do que o saldo de resgate, significa que o mercado está "derretendo" os ativos.

In [9]:
import plotly.express as px

fig = px.bar(analise_result, x='ANO_MES', y='SALDO_LIQUIDO',
             title='Fluxo de Caixa da Indústria de Fundos (Captação Líquida)',
             labels={'SALDO_LIQUIDO': 'Saldo (Captação - Resgate)', 'ANO_MES': 'Mês/Ano'},
             color='SALDO_LIQUIDO',
             color_continuous_scale=px.colors.diverging.RdYlGn) # Verde para positivo, vermelho para negativo

fig.show()

3. Por tipo de Fundo

In [10]:
# Analisando quem dominou o otimismo de Janeiro
gold_market_share = df_silver.groupBy("TP_FUNDO_CLASSE") \
    .agg(
        F.sum("VL_PATRIM_LIQ").alias("PL_TOTAL"),
        F.sum(F.col("CAPTC_DIA") - F.col("RESG_DIA")).alias("SALDO_LIQUIDO"),
        F.countDistinct("CNPJ_FUNDO_CLASSE").alias("QTD_FUNDOS")
    ).orderBy(F.desc("PL_TOTAL"))

# Visualizando o resultado
df_p = gold_market_share.toPandas()
display(df_p)

# Gráfico de Barras por Classe
import plotly.express as px
px.bar(df_p, x='TP_FUNDO_CLASSE', y='SALDO_LIQUIDO',
       title='Saldo Líquido por Classe de Fundo (Jan/2024)',
       color='SALDO_LIQUIDO', text_auto='.2s').show()

,TP_FUNDO_CLASSE,PL_TOTAL,SALDO_LIQUIDO,QTD_FUNDOS
0,FI,2.410360e+14,1.097480e+11,25960
1,FAPI,1.864845e+10,-1.873671e+06,13


4. Market Share por Classe de Fundo

Qual tipo de fundo domina o mercado? (Ações, Renda Fixa, Multimercado?). Essa análise mostra onde o dinheiro está concentrado.

In [15]:
from pyspark.sql.window import Window

# Calculando Market Share com a correção da Window
gold_market_share = df_silver.groupBy("TP_FUNDO_CLASSE") \
    .agg(
        F.sum("VL_PATRIM_LIQ").alias("PL_POR_CLASSE"),
        F.countDistinct("CNPJ_FUNDO_CLASSE").alias("QTD_FUNDOS")
    ).withColumn("PERCENTUAL_PL", F.col("PL_POR_CLASSE") / F.sum("PL_POR_CLASSE").over(Window.partitionBy())) \
    .orderBy(F.desc("PL_POR_CLASSE"))

gold_market_share.show()

+---------------+--------------------+----------+--------------------+
|TP_FUNDO_CLASSE|       PL_POR_CLASSE|QTD_FUNDOS|       PERCENTUAL_PL|
+---------------+--------------------+----------+--------------------+
|             FI|2.410360216115541E14|     25960|  0.9999226381032803|
|           FAPI|1.864844648885000...|        13|7.736189671977602E-5|
+---------------+--------------------+----------+--------------------+



5. Ranking de Eficiência (Top 10 Captação Líquida)

Não basta ter patrimônio alto, queremos saber quem está atraindo novos investidores. Vamos calcular o saldo entre entradas e saídas.

In [13]:
df_captacao = df_silver.withColumn("CAPTC_LIQUIDA", F.col("CAPTC_DIA") - F.col("RESG_DIA"))

gold_top_captadores = df_captacao.groupBy("CNPJ_FUNDO_CLASSE") \
    .agg(F.sum("CAPTC_LIQUIDA").alias("SALDO_CAPT_LIQUIDA")) \
    .orderBy(F.desc("SALDO_CAPT_LIQUIDA")).limit(10)

6. Análise de Volatilidade de Patrimônio

Fundos com patrimônio que oscila muito podem indicar riscos. Vamos medir o desvio padrão do Patrimônio Líquido por fundo.

In [14]:
gold_volatilidade = df_silver.groupBy("CNPJ_FUNDO_CLASSE") \
    .agg(F.stddev("VL_PATRIM_LIQ").alias("VOL_PATRIMONIO")) \
    .orderBy(F.desc("VOL_PATRIMONIO")).limit(10)

###Salvando as tabelas na Gold

In [16]:
# Dicionário para facilitar o salvamento em lote
tabelas_gold = {
    "ranking_patrimonio": ranking_patrimonio,
    "market_share": gold_market_share,
    "top_captadores": gold_top_captadores,
    "volatilidade": gold_volatilidade
}

for nome, df in tabelas_gold.items():
    path = f"/content/drive/MyDrive/pipeline_bigdata_pyspark/gold/{nome}"
    df.write.mode("overwrite").parquet(path)
    print(f"✅ Tabela '{nome}' salva com sucesso em {path}")

✅ Tabela 'ranking_patrimonio' salva com sucesso em /content/drive/MyDrive/pipeline_bigdata_pyspark/gold/ranking_patrimonio
✅ Tabela 'market_share' salva com sucesso em /content/drive/MyDrive/pipeline_bigdata_pyspark/gold/market_share
✅ Tabela 'top_captadores' salva com sucesso em /content/drive/MyDrive/pipeline_bigdata_pyspark/gold/top_captadores
✅ Tabela 'volatilidade' salva com sucesso em /content/drive/MyDrive/pipeline_bigdata_pyspark/gold/volatilidade


###Plotagem

In [17]:
import plotly.express as px

df_p = gold_market_share.toPandas()
fig1 = px.pie(df_p, values='PL_POR_CLASSE', names='TP_FUNDO_CLASSE',
             title='Distribuição de Patrimônio Líquido por Classe (Jan/2024)',
             hole=0.3)
fig1.show()

A análise de Market Share revela que a classe FI (Fundo de Investimento) detém a hegemonia absoluta do patrimônio em Janeiro de 2024, indicando uma alta concentração de capital em instrumentos tradicionais.



In [18]:
df_c = gold_top_captadores.toPandas()
fig2 = px.bar(df_c, x='CNPJ_FUNDO_CLASSE', y='SALDO_CAPT_LIQUIDA',
             title='Top 10 Fundos por Captação Líquida',
             color='SALDO_CAPT_LIQUIDA', text_auto='.2s')
fig2.show()

A análise de Eficiência de Captação identifica os 10 fundos que apresentaram a maior resiliência e atratividade comercial em Janeiro de 2024. O destaque vai para os fundos que, mesmo num mercado de triliões, conseguiram manter um Saldo Líquido (Captação - Resgate) positivo e robusto. Esta métrica é um indicador de confiança do investidor: enquanto o património total (PL) reflete o passado, a Captação Líquida reflete o momento presente e a expectativa de performance futura da gestora.

In [19]:
df_v = gold_volatilidade.toPandas()
fig3 = px.scatter(df_v, x='CNPJ_FUNDO_CLASSE', y='VOL_PATRIMONIO',
                 size='VOL_PATRIMONIO', color='CNPJ_FUNDO_CLASSE',
                 title='Top 10 Fundos com Maior Volatilidade de Patrimônio')
fig3.show()

A análise de volatilidade revela os fundos que apresentaram as maiores oscilações patrimoniais durante o mês de janeiro de 2024. Notamos que os líderes deste ranking não são necessariamente os maiores em patrimônio total, o que indica que a volatilidade está mais atrelada à movimentação intensa de resgates e aportes (giro de cotistas) do que ao tamanho do fundo em si. Para o investidor, este gráfico serve como um mapa de risco operacional, identificando veículos que, apesar de estarem em uma classe estável como 'FI', possuem um comportamento de liquidez muito mais agressivo e instável.